# API communication with the chosen LLM 
This notebook could be moved in the prompt generator

In [11]:
import openai
from openai import OpenAI
import json
import os

client = OpenAI(
    api_key = 'API-KEY'
)

In [12]:
def extract_claims_from_html(table_html, context):
    prompt = f"""
    Consider the claims being in the form <{{Specification, Specification, …}}, Measure, Outcome>

    Claims must be extracted according to the following format:
    <{{Specification, Specification, …}}, Measure, Outcome>

    Specification: |name, value| pair describing the details of an experiment
    E.g.: |dataset, Spider|, or |LLM, Llama27b|
    Measure: metric or measure used to evaluate the experiment
    E.g.: F1-measure
    Outcome: outcome value related to metric
    E.g.: 0.89

    Extract all the claims, even with empty value for the measure in that exact form from the following html table:
    {table_html}

    Know that the context where the table was mentioned is the following:
    {context}
    """

    try:
        # Nuovo metodo per chiamare il completamento
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
        )
        # Estrai il contenuto della risposta
        return response["choices"][0]["message"]["content"].strip()

    except openai.APIConnectionError as e:
        print(f"Errore di connessione all'API: {e}")
    except Exception as e:
        print(f"Errore generico: {e}")
    return None

In [13]:
def process_json_file(json_file):
    # Leggi il file JSON
    with open(json_file, 'r') as f:
        data = json.load(f)

    for table_id, table_info in data.items():
        caption = table_info.get("caption", "")
        table_html = table_info.get("table", "")
        footnotes = table_info.get("footnotes", [])
        references = table_info.get("references", [])

        # Crea il contesto completo con la didascalia, le note e i riferimenti
        context = f"Caption: {caption}\nFootnotes: {', '.join(footnotes)}\nReferences: {', '.join(references)}"

        print(table_html)
        print(context)
        # Estrai i claim dalla tabella
        claims = extract_claims_from_html(table_html, context)

        # Stampa o salva i claim estratti
        print(f"Claims for table {table_id}:")
        print(claims)
        print("\n" + "="*50 + "\n")

In [14]:
# Specifica il percorso del tuo file JSON nella cartella 'selected_tables'
json_file = 'selected_tables/2308.12519.json'
process_json_file(json_file)

[['<td class="ltx_td ltx_align_left ltx_border_r ltx_border_tt" id="S5.T2.fig1.1.1.1">\n          Model\n         </td>\n         \n', '<td class="ltx_td ltx_align_right ltx_border_tt" id="S5.T2.fig1.1.1.2">\n          Pass Rate (%)\n         </td>\n        \n'], ['<td class="ltx_td ltx_align_left ltx_border_r ltx_border_t" id="S5.T2.fig1.1.2.1">\n          CoT\n         </td>\n         \n', '<td class="ltx_td ltx_align_right ltx_border_t" id="S5.T2.fig1.1.2.2">\n          16.60\n         </td>\n        \n'], ['<td class="ltx_td ltx_align_left ltx_border_r" id="S5.T2.fig1.1.3.1">\n          CoT@3\n         </td>\n         \n', '<td class="ltx_td ltx_align_right" id="S5.T2.fig1.1.3.2">\n          31.20\n         </td>\n        \n'], ['<td class="ltx_td ltx_align_left ltx_border_r" id="S5.T2.fig1.1.4.1">\n          Reflexion\n         </td>\n         \n', '<td class="ltx_td ltx_align_right" id="S5.T2.fig1.1.4.2">\n          26.60\n         </td>\n        \n'], ['<td class="ltx_td ltx_ali

KeyboardInterrupt: 